# Regular expressions for data cleanup

If data isn't formatted in a usable way, it doesn't do us any good. That's where cleanup comes in.

The most time-consuming part of most analysis is getting data into a useful format. By comparison, performing the actual interrogations is much less time-consuming.

In the first section, I'll cover the difference between data that looks right to humans and data that the machine understands.

In the second section, I'll use regular expressions to develop a function to convert height strings to inches, with an emphasis on the process of developing code.

In the third part, I'll demonstrate how to apply the conversion function to the data.

# Part 1: Looks good to me, HAL, I don't know what you're complaining about

The goal for us is to have data in a format we can use. So we might want to convert strings to numbers, but why is that important?

In python, 2 is number and "2" is a character. They are different types of data, but we can still perform math on them:


In [8]:
two_character = "2"
two_number = 2

def double(x):
    return 2 * x

print double(two_character)
print double(two_number)
print double("Any text. ")

22
4
Any text. Any text. 


### "Expected" behavior

The above example demonstrates what might be considered unexpected behavior:

    "2" * 2 = "22" 
    2 * 2 = 4
    
That's because Python's binary + and * operators work on both strings and numbers as long as both operands are the same type. But if you try to add a string and a number together, you get an error:

In [9]:
print 2 + "2"

TypeError: unsupported operand type(s) for +: 'int' and 'str'

This might not seem like a problem, and in fact, as long as you understand this behavior, it's not. It's just a design decision the developers of Python made. They could have made separate operators for string concatenation and addition, but they didn't.

So when could this be a problem?

Let's say you import a spreadsheet and it ends up like it's full of numbers:

In [32]:
import pandas as pd

df = pd.DataFrame([[1,2,3,4,16],["1","2","3","4","F"]]
                  ,index=["CT","NY"])
df

,0,1,2,3,4
CT,1,2,3,4,16
NY,1,2,3,4,F


Looks good, right? OK, let's try and use it for data analysis. Let's multiply every value by 10. (Don't worry about that weird lambda thing. If you want to learn more about it check out this article (http://www.secnetix.de/olli/Python/lambda_functions.hawk) also linked to in the further reading section.)

In [33]:
df.apply(lambda x : x * 10)

,0,1,2,3,4
CT,10,20,30,40,160
NY,1111111111,2222222222,3333333333,4444444444,FFFFFFFFFF


### Uh oh!

Even though these numeric characters in the "NY" row look like numbers, they aren't. In this case we could spot the problem results by eye. But what if the results looked more legit?

In [34]:
df.apply(lambda x : x * 2)

,0,1,2,3,4
CT,2,4,6,8,32
NY,11,22,33,44,FF


We can check the type of data in a column another way, using .dtypes:

In [35]:
df.dtypes

0    object
1    object
2    object
3    object
4    object
dtype: object

That "object" type for a column means Pandas doesn't think the data is numeric.

So, before we start to do the analysis we need to clean up the data. In this case, Pandas can handle converting the NY column to numeric values with the .to_numeric method:

In [36]:
df.loc["NY"] = pd.to_numeric(df.loc["NY"])
df

ValueError: Unable to parse string

### Why the error?

The .to_numeric() method failed. It couldn't convert "F" to a number, so it raised an exception. We have to tell Pandas how to handle values it can't convert to a number by using the optional errors argument:

In [46]:
df.loc["NY"] = pd.to_numeric(df.loc["NY"]
                             ,errors="coerce")
df

,0,1,2,3,4
CT,10,20,30,40,160
NY,11,21,31,41,NaN


The table mostly looks the same, except the "F" is now a null value. Now let's try again to multiply all the values by 10:

In [38]:
df = df.apply(lambda x: x * 10)
df

,0,1,2,3,4
CT,10,20,30,40,160
NY,10,20,30,40,NaN


Yay! The calculation worked on as expected on all the fields. Let's verify that we're working with numbers, not "objects":

In [12]:
df.dtypes

0    float64
1    float64
2    float64
3    float64
4    float64
dtype: object

That's good!

float64 means Pandas sees the values in the column as valid floating decimal numbers. Having that NaN doesn't spoil the whole column. It's just like having a blank cell in a spreadsheet, which is OK.

# Part 2: Sizing up the matter

In this section we'll import a dataset (on stun gun use by CT police) and convert some ugly data to useful data.

### Importing the data

Start by importing the spreadsheet. I'm paring it down to just a few columns. I'll use the .head() method to visually inspect the first few rows in the table. Then I'll use .dtypes to see what data type Pandas inferred when importing:

In [17]:
stuns = pd.read_excel("data/2015 Reported Taser Data.xlsx")\
[["Incident Case Number","Height","Weight"]]
stuns.head()

,Incident Case Number,Height,Weight
0,NaN,NaN,NaN
1,2015-055,"5'10""",220
2,15-19,"5'8""",150
3,15-00010,"5'10""",175
4,1500006763,"5'10""",150


In [14]:
stuns.dtypes

Incident Case Number    object
Height                  object
Weight                  object
dtype: object

All objects! That's not good. 

We can tell that the case number can be any kind of string.

Height, that looks like a problem. The commonly use prime notation for feet and inches we'll get to. 

For now, let's tackle the weight. Why is that column not numeric?

In [18]:
stuns["Weight"]

0          NaN
1          220
2          150
3          175
4          150
5          153
6          260
7          200
8          177
9          120
10         155
11         229
12         160
13         175
14         110
15         200
16         170
17         180
18         190
19         190
20         250
21         235
22         205
23         250
24         185
25         209
26         160
27         200
28         230
29         180
        ...   
581        245
582        270
583        170
584        145
585        180
586        240
587        145
588        115
589        350
590        180
591        185
592    Unknown
593        220
594        300
595        220
596    Unknown
597        200
598        160
599        300
600        170
601        165
602        190
603        195
604        160
605        155
606        170
607        215
608        280
609    Unknown
610        215
Name: Weight, dtype: object

There are some fields that have the word "Unknown" in them instead of a number. No problem. Let's convert those all to nulls, which is really just the computer word for "Unknown":

In [19]:
stuns["Weight"] = pd.to_numeric(stuns["Weight"], 
                                errors="coerce")
stuns.dtypes

Incident Case Number     object
Height                   object
Weight                  float64
dtype: object

Good, it now sees the column as a float. Let's make sure we can do calculations on it. Let's try and find the average weight of stun gun subjects.

In [21]:
stuns["Weight"].mean()

188.83273381294964

Great. Now on to the more complicated height conversion


### The approach

Pandas works great for simple conversions, but it doesn't know how to convert 5'6" to some decimal value. We'll have to do it ourselves.

First of all, let's choose a desired result. We could represet 5'6" as 5.5 feet, or 66 inches. Since it's not natural read 5.5 and know it means 5 feet 6 inches let's go with inches.

We'll start by writing a function that can do the conversion from prime-notation to inches, then apply that function over the entire column.



### Writing the 'spec' for the inches function

We will accept and convert strings that match the following patterns:

1. Strings with both a feet and inches value, denoted with a prime and double prime respectively, like: 5'6"
2. The same as 1 but missing the trailing double prime mark: 5'6
3. Strings with only a feet value, like: 5"

Anything else, we'll return null, which in Python is None.

Let's add a test case for each format we accept and a couple that should fail, returning None


### Developing and testing the inches function
I'm going to write this iteratively so you can see my development process. Start by making a function and a test method that calls the method with several inputs. In this case all the inches does for now is return the string you give it. That's sometimes called a code stub or starter code.

In [22]:
# Convert prime-notation height to inches
def inches(prime_str):
    return prime_str

def test_inches():
    # Should return 66
    print "5'6\" Should return 66:\t\t", inches("5'6\"")
    # Should return 66
    print "5'6 Should return 66:\t\t",inches("5'6")
    # Should return 60
    print "5' Should return 60:\t\t",inches("5'")
    # Should return None
    print "5 feet ... Should return None:\t"
    ,inches("5 feet ten inches")
    print "5'6' Should return None:\t",inches("5'6'")
    print "5'F\" Should return None:\t",inches("5'F'")
    print "5'6LOL Should return None:\t",inches("5'6\"LOL")
    print "5'120 Should return None:\t",inches("5'120")

    
test_inches()

5'6" Should return 66:		5'6"
5'6 Should return 66:		5'6
5' Should return 60:		5'
5 feet ... Should return None:	5 feet ten inches
5'6' Should return None:	5'6'
5'F" Should return None:	5'F'
5'6LOL Should return None:	5'6"LOL
5'120 Should return None:	5'120


So we've defined a pattern and set up some basic tests. 

It's time to write the code to extract data from strings that match our specified patterns. The tool used to do this is regular expressions, which are not unique to Python. 

We'll use the regular expressions (re) module.

In [24]:
# We're going to need regular expressions now;
# import the re module
import re

# Convert prime-notation height to inches
def inches(prime_str):
    # Extract feet and inches from the string, as long as 
    # 1. feet is a numeric value at the start of the string, followed
    # by a single prime mark
    # 2. inches is a numeric value preceded by the feet and single prime
    # mark and optionally followed by a double prime mark
    result = re.match(r"(?P<feet>[0-9]+)'(?P<inches>[0-9]{0,2}?)[\"]?\Z", prime_str)
    
    # Access the extract values using the .group() method
    # We'll print the values out but not do anything with them yet
    feet = result.group("feet")
    inches = result.group("inches")
    print "\nFeet: ", feet 
    print "Inches: ", inches

    return prime_str
    
test_inches()

5'6" Should return 66:		
Feet:  5
Inches:  6
5'6"
5'6 Should return 66:		
Feet:  5
Inches:  6
5'6
5' Should return 60:		
Feet:  5
Inches:  
5'
5 feet ... Should return None:	

AttributeError: 'NoneType' object has no attribute 'group'

We don't want to throw an exception when the pattern match fails. We just want to return None.

We'll use try and except to handle the exception and return None.


In [25]:
# We're going to need regular expressions now;
# import the re module
import re

# Convert prime-notation height to inches
def inches(prime_str):
    ## Wrap the function in try/except
    try: 
        # Extract feet and inches from the string, as long as 
        # 1. feet is a numeric value at the start of the string, followed
        # by a single prime mark
        # 2. inches is a numeric value preceded by the feet and single prime
        # mark and optionally followed by a double prime mark
        result = re.match(r"(?P<feet>[0-9]+)'(?P<inches>[0-9]{0,2}?)[\"]?\Z", prime_str)
    
        # Access the extract values using the .group() method
        # We'll print the values out but not do anything with them yet
        feet = result.group("feet")
        inches = result.group("inches")
        print "\nFeet: ", feet 
        print "Inches: ", inches
    except:
        # This code will only be called if the code within the try: block
        # throws an exception
        return None

    # We won't get to this point unless the code in the try block was successful
    return prime_str
    
test_inches()

5'6" Should return 66:		
Feet:  5
Inches:  6
5'6"
5'6 Should return 66:		
Feet:  5
Inches:  6
5'6
5' Should return 60:		
Feet:  5
Inches:  
5'
5 feet ... Should return None:	None
5'6' Should return None:	None
5'F" Should return None:	None
5'6LOL Should return None:	None
5'120 Should return None:	None


OK, now we're getting somewhere. It's time to take the feet and inches and convert them to inches.

The algorithm is:

    12 * feet + inches

In [28]:
# We're going to need regular expressions now;
# import the re module
import re

# Convert prime-notation height to inches
def inches(prime_str):
    ## Wrap the function in try/except
    try: 
        # Extract feet and inches from the string, as long as 
        # 1. feet is a numeric value at the start of the string, followed
        # by a single prime mark
        # 2. inches is a numeric value preceded by the feet and single prime
        # mark and optionally followed by a double prime mark
        result = re.match(r"(?P<feet>[0-9]+)'(?P<inches>[0-9]{0,2}?)[\"]?\Z", prime_str)
    
        # Access the extract values using the .group() method
        feet = result.group("feet")
        inches = result.group("inches")
        # print "\nFeet: ", feet 
        # print "Inches: ", inches, "\n"
    except:
        # This code will only be called if the code within the try: block
        # throws an exception
        return None

    # We won't get to this point unless the code in the try block was successful
    
    # Convert feet and inches to inches
    feet = result.group("feet")
    inches = result.group("inches")
    return feet * 12 + inches
    
test_inches()

5'6" Should return 66:		5555555555556
5'6 Should return 66:		5555555555556
5' Should return 60:		555555555555
5 feet ... Should return None:	None
5'6' Should return None:	None
5'F" Should return None:	None
5'6LOL Should return None:	None
5'120 Should return None:	None


Oh no! Either that person is really tall or it looks like the feet and inches returned from the regular expression aren't being treated like numbers.

Let's use the int() method to convert the values to numbers.


In [42]:
# We're going to need regular expressions now;
# import the re module
import re

# Convert prime-notation height to inches
def inches(prime_str):
    ## Wrap the function in try/except
    try: 
        # Extract feet and inches from the string, as long as 
        # 1. feet is a numeric value at the start of the string, followed
        # by a single prime mark
        # 2. inches is a numeric value preceded by the feet and single prime
        # mark and optionally followed by a double prime mark
        result = re.match(r"(?P<feet>[0-9]+)'(?P<inches>[0-9]{0,2}?)[\"]?\Z", prime_str)
    
        # Access the extract values using the .group() method
        feet = int(result.group(1))
        if result.group(2) == "":
            inches = 0
        else:
            inches = int(result.group(2))
        # print "Feet: ", feet 
        # print "Inches: ", inches
    except:
        # This code will only be called if the code within the try: block
        # throws an exception
        return None

    # We won't get to this point unless the code in the try block was successful
        
    return feet * 12 + inches
    
test_inches()
#inches("5'6\"")
#inches("5'F")

5'6" Should return 66:		66
5'6 Should return 66:		66
5' Should return 60:		60
5 feet ... Should return None:	None
5'6' Should return None:	None
5'F" Should return None:	None
5'6LOL Should return None:	None
5'120 Should return None:	None


### Reading the regex

OK, it seems to be working. But let's break down this line:
   
    result = re.match(r"(?P<feet>[0-9]+)'(?P<inches>[0-9]{0,2}?)[\"]?\Z", prime_str)
    
It looks scary. I understand. For years, if I saw a line like that, I would ignore it and find some other way to solve my problem. The key to reading regex is to recognize its components.
    
Let's start by ignoring the `result = ...` assignment part and focus on `re.match...` part.

We make a call to `re.match(expression, string)` to look for `expression` in string where `expression` is:

    r"(?P<feet>[0-9]+)'(?P<inches>[0-9]{0,2}?)[\"]?\Z"
    
and `string` is `prime_str`, the argument passed to the inches function

The `re.match ()` function  is described in detail here (https://docs.python.org/2/library/re.html#re.match). It looks for `expression` in `string`. Let's break down the string.

1. The `r` signifies that the rest of the expression will be in raw notation (https://docs.python.org/2/library/re.html#raw-string-notation), eliminating the need to use a silly amount of backlash escape characters. 
2. The next chunk is `(?P<feet>[0-9]+)`. The parentheses signify that everything inside them should be extracted as a "group" we can access later. Inside it, we see `?P<feet>`, which stores the group under a symbolic name we can use to access it later. The actual pattern portion is `[0-9]+` The brackets signify that any character is a valid match. In this case, the numerals 0-9 are all valid. The `+` signifies that there needs to be at least one of these values.
3. The next chunk is the `'` prime mark (actually a single quote). There has to be one.
4. The next chunk is `(?P<inches>[0-9]{0,2}?)`. It looks a lot like the chunk to extract feet. The only differences are that we are storing the result under a different symbolic name "inches" and instead of `+` we have `{0,2}?`. Remember the `+` means look for at least one occurrence, but we don't want to require inches. Instead the `?` means to look for exactly 0 or 1 occurrences of the `[0-9]` pattern (any numeral). Putting `{0,2}?` means look for 0 to 2 numerals, but not more. That's because we don't want to accept strings like 5'120.
5. The next chunk `[\"]?` looks for zero or one double prime (double quotation mark), since the trailing double prime is optional according to our spec.
6. Finally, the `\Z` matches the end of the string. We don't want to allow any trailing characters, like 5'6LOL

# Part 3: Converting all the data

Now that we have our handy inches() function, we need a way to apply it to every value in the Height column.

Pandas has a method called `.apply()` which lets us do just that (http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.Series.apply.html).

It's a special kind of function called a higher order function, so-called because it takes another function as its argument. The apply function will take each value in the Series (column) and replace it with whatever value the function returns.

Let's dredge up our example dataframe:

In [39]:
df

,0,1,2,3,4
CT,10,20,30,40,160
NY,10,20,30,40,NaN


Let's say we want to convert every value in the "NY" row to 1. We can write a function called one() that only returns the value 1, then call the `.apply()` method:

In [40]:
# We accept argument x even though we do nothing with it
# because the function passed as an argument to apply()
# must accept a single argument
def add_one(x):
    return x + 1

df.loc["NY"] = df.loc["NY"].apply(add_one)
df

,0,1,2,3,4
CT,10,20,30,40,160
NY,11,21,31,41,NaN


So now let's do apply our inches method to our Height column in the stuns DataFrame and assign it to a new column.

In [43]:
stuns["Heights_numeric"] = stuns["Height"].apply(inches)
stuns.head()

,Incident Case Number,Height,Weight,Heights_numeric
0,NaN,NaN,NaN,NaN
1,2015-055,"5'10""",220,70
2,15-19,"5'8""",150,68
3,15-00010,"5'10""",175,70
4,1500006763,"5'10""",150,70


Looks like it worked! Finally let's look for the mean and median height:

In [44]:
stuns["Heights_numeric"].mean()

69.754385964912274

# That's all, folks!

So by now you should have an understanding of:
1. why we can't trust our eyeballs to validate data; 
2. how to develop a conversion function iteratively and test it along the way; 
3. how to use regular expressions to convert strings that match a known pattern


# Further reading


* Python documentation on regular expressions module, https://docs.python.org/2/library/re.html
* Python lambda functions: http://www.secnetix.de/olli/Python/lambda_functions.hawk